# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/stepvideo-t2v',
 '/microsoft/wham',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/SakanaAI/AI-CUDA-Engineer-Archive',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
B

In [21]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nperplexity-ai/r1-1776\nUpdated\n6 days ago\n•\n12.2k\n•\n1.7k\ndeepseek-ai/DeepSeek-R1\nUpdated\n1 day ago\n•\n4.52M\n•\n10.2k\nmicrosoft/OmniParser-v2.0\nUpdated\n7 days ago\n•\n5.31k\n•\n947\nstepfun-ai/stepvideo-t2v\nUpdated\n6 days ago\n•\n1.3k\n•\n356\nmicrosoft/wham\nUpdated\n4 days ago\n•\n193\nBrowse 1M+ models\nSpaces\nRunning\n1.55k\n1.55k\nThe Ultra-Scale Playbook\n🌌\nThe ultimate g

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of the AI community, dedicated to building the future of machine learning. Our platform serves as a collaborative space where developers, researchers, and enterprises come together to share and refine models, datasets, and applications. This commitment to collaboration has established Hugging Face as the home of machine learning innovation.

- **Website:** [huggingface.co](https://huggingface.co)
- **Core Mission:** To enhance and democratize access to machine learning resources and foster collaboration among the ML community.

## What We Offer
### Models
With over **1 million models** available, Hugging Face is your ultimate destination for machine learning resources. Popular models include:
- perplexity-ai/r1-1776
- deepseek-ai/DeepSeek-R1
- microsoft/OmniParser-v2.0

### Datasets
Access and share from a vast library of over **250,000 datasets**, designed for various ML tasks. Our community keeps them updated to ensure the highest quality available.

### Spaces
Run and collaborate on applications seamlessly with our **Spaces** platform. Whether you’re interested in deploying models or engaging with AI applications, Hugging Face provides a user-friendly interface to manage your projects.

## Our Customers
Hugging Face is trusted by more than **50,000 organizations**, including industry leaders:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Intel**
- **Microsoft**
These collaborations enable a continuous flow of ideas and innovation, driving the future of AI.

## Company Culture
At Hugging Face, we pride ourselves on fostering a community-centric culture that values:
- **Collaboration:** We believe in the power of open source and community engagement.
- **Innovation:** Our work is driven by curiosity and a relentless pursuit of improvement.
- **Inclusivity:** We aim to create a welcoming environment for individuals from diverse backgrounds.

## Careers
Join our mission! We are constantly on the lookout for talent passionate about AI and ML. Available positions span various areas, from engineering to marketing, and beyond. At Hugging Face, you’ll have the opportunity to work in a supportive environment that values growth, creativity, and impact.

### Explore Job Opportunities
Check our careers page for open positions and take the next step in your journey with us!

## Connect with Us
Stay engaged with our community through various platforms:
- **Social Media:** [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), [Discord](https://discord.gg/huggingface)
- **GitHub:** [huggingface](https://github.com/huggingface)

---

**Join Hugging Face** and be part of the AI community that is shaping the future!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'social media - LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'social media - Twitter', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face: Building the Future of AI

## Who We Are
Hugging Face is a vibrant AI community dedicated to fostering innovation and collaboration in the machine learning space. Our platform serves as the essential hub where machine learning practitioners come together to collaborate on models, datasets, and applications, making strides in AI technology and enabling groundbreaking advancements.

**Explore AI Apps or Browse 1M+ Models!**

## Our Offerings
### Models
With over 1 million models available, Hugging Face is the go-to source for machine learning enthusiasts and professionals. Some trending models include:
- **perplexity-ai/r1-1776**
- **deepseek-ai/DeepSeek-R1**
- **microsoft/OmniParser-v2.0**

### Datasets
Access a diverse collection of over 250,000 datasets that can be utilized for various machine learning tasks. Highlighted datasets include:
- **Congliu/Chinese-DeepSeek-R1-Distill-data-110k**
- **facebook/natural_reasoning**

### Spaces
Collaborate and showcase your applications using our Spaces feature, which hosts various running applications in an interactive manner.

## Our Culture
At Hugging Face, we promote a culture of openness, collaboration, and continuous learning. We believe in the power of community and the collective intelligence that emerges when creative minds work together. Our commitment to transparency and open source software nurtures an ecosystem where everyone can contribute and grow.

## Customers
Join the ranks of over 50,000 organizations using Hugging Face, including industry leaders like:
- **Google**
- **Microsoft**
- **Amazon Web Services**
- **Intel**

Whether you are an enterprise seeking advanced AI solutions or an individual passionate about machine learning, we welcome you to join our thriving community.

## Careers at Hugging Face
We are constantly on the lookout for passionate individuals to help us shape the future of AI. Working at Hugging Face means:
- Engaging in meaningful work with a talented team
- Pioneering collaborative projects that push the boundaries of machine learning
- Contributing to open-source technologies that benefit the global community

If you are ready to make an impact in the AI domain, explore our current job openings and be part of our journey.

## Let’s Innovate Together!
Whether you are an AI researcher, a data scientist, or someone looking to venture into the world of machine learning, Hugging Face welcomes you. Join us as we build the future of AI together.

[Explore Hugging Face](https://huggingface.co) | [Join us on GitHub](https://github.com/huggingface) | [Follow us on Twitter](https://twitter.com/huggingface) 

#### Hugging Face – The AI community building the future!

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face: Where AI Gets a Hug!

---

**Who Are We?**
At Hugging Face, we're not just building models, we're crafting the *future* of AI! Imagine a community of machine-learning experts collaborating with the enthusiasm of kids in a candy store! 🤗

---

**What Do We Do?**
- **Models**: With over **1M+ models**, our platform is the buffet of AI! From perplexity to deep-seeking, we have experimented so much you'll feel like a mad scientist (without the lab coat).
- **Datasets**: Don't let your AI starve! We offer **250k+ datasets** to keep your algorithms fueled and happy.
- **Spaces**: Space is limited... said no one at Hugging Face! With **1.55k Spaces**, you can play (and deploy) with AI like you’re building a LEGO city – but for code!

---

**Caring is Our Superpower: Company Culture!**
At Hugging Face, we believe in collaboration, celebration, and making the workplace as cozy as your grandma's living room. We’re not just colleagues – we’re a family! 🥳 

Expect:
- **Daily AI Jam Sessions**: Bring your best model or dataset and jam with fellow data wizards!
- **AI-based Snack Recommendations**: Because who doesn’t want to know which snack pairs best with that new transformer model you’ve been working on? 🍿
- **Pawtastic Days**: Bring your furry friends to work (virtually, of course!). Because what's AI without some cuteness! 🐶

---

**Our Customers 🚀**
Join the ranks of over **50,000 organizations**, including giants like Google, Microsoft, and Grammarly! We provide them the tools to make their AI dreams a reality, all while keeping their humor intact. 

---

**Careers: Join the Hugging Face Family!**
Are you clever, fun, and proficient in either AI or puns? If yes, then we WANT you! Positions are available in:
- **Machine Learning Sorcery**
- **Data Wizardry** 
- **Space Exploration** (a.k.a. cloud solutions)
- **Model Therapy** (we offer emotional support for neural networks)

**Why Hugging Face?**
- A workplace where every day feels like *bring your best model to work day*.
- Lunch breaks filled with spirited debates on which model performs best: Transformers or… well, Transformers (we're a little biased).
- Opportunities to grow both your skills and your pun game! 

---

**Ready to Join Us?**
Visit [Hugging Face Careers](https://huggingface.co/careers) and let’s create some AI magic together! 

---

Whether you’re looking for groundbreaking AI knowledge or an engaging workplace culture, remember: **at Hugging Face, we care about models and hugs equally**. 🤗

---


In [24]:
stream_brochure("Arturo Jiménez", "https://www.arturo-jimenez.es")

Found links: {'links': [{'type': 'about page', 'url': 'https://arturo-jimenez.es/sobre-mi/'}, {'type': 'contact page', 'url': 'https://arturo-jimenez.es/contacto/'}]}



# Welcome to Arturo Jiménez: The Tech Barrier Busters!

### 🚀 **Who Are We?**
At **Movilok**, we don’t just develop apps; we set out to **dismantle technological barriers**, one pixel at a time! Founded in 2002 (back when 'mobile app' was just a funny phrase kids used), we’ve been leading the charge into the exciting realm of mobile applications and cutting-edge software projects. 

### 🤖 **What We Do:**
- **Unlocking Imagination:** Got an idea for a mobile application? We’ll help you unlock it faster than you can say "download now!"
- **Integrating Legacy Systems:** If your tech feels like it needs a time machine, we’re your go-to team to connect it to the current age.
- **Fast-Tracking Innovation:** It’s our job to keep up with the latest tech trends, which is tougher than keeping up with Kardashian family drama!

### 🎉 **Our Star-Studded Clients:**
We’ve partnered with some **BIG NAMES** like Cepsa, Bankinter, Mutua Madrileña, and BBVA. Our apps are so good, they’ve even won awards (which we’ve neatly displayed in the imaginary trophy cabinet).

### 💻 **Our Superheroes (Team Culture):**
Our team is a mix of:
- Developers who see code as poetry 🎤
- Designers who believe *everything* is an art project 🎨
- Project managers who can juggle flaming swords while timing Gantt charts ⏰🔥

We value flexibility, innovation, and a hearty dose of fun. Because if you're not having fun, are you even working in tech?!

### 🌍 **Careers at Movilok:**
Are you ready to join us? We’re searching for:
- Future code Jedi 🥷
- Creative idea ninjas 💡
- Project wizards 🧙‍♂️

If you can wield a keyboard as well as Thor wields his hammer, then we want you! 

### 🤝 **Customers for Life... Literally!**
We don’t just serve businesses; we serve *legacies*. Our tech solutions truly integrate into our client’s operations. (Don’t worry, we won’t suggest stitched-together band-aids for Real-World Problems 😅).

### **A Peek Into Our Minds:**
In our blog, we share juicy stories about software development and the latest in AI, because who needs Netflix when you have code to analyze?

### 📞 **Get in Touch!**
Got an idea, a question, or a mildly-interesting joke? We’d love to hear from you. Just hit that “Contact” button and let’s start creating tech magic together!

---

### ✨ **Join us at Movilok** – Where tech dreams come true and innovation is just the cherry on top of an already fantastic sundae! 🍨



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>